In [ ]:
import pandas as pd

In [ ]:
Dim_customer=pd.read_csv('/content/Dim_customer.csv')
Dim_currency=pd.read_csv('/content/Dim_currency.csv')
Fact_invoice=pd.read_csv('/content/Fact_invoice.csv')

In [ ]:
top_customer=Fact_invoice.groupby("customerid")["total"].sum().sort_values(ascending=False).head(5).reset_index() #Get the customer that spent the most money
top_customer

,customerid,total
0,59,71.31
1,132,65.36
2,179,55.44
3,6,49.62
4,26,47.62


In [ ]:
Dim_currency['invoicedate']=pd.to_datetime(Dim_currency['invoicedate']).dt.tz_localize(None) #Change the Date format to match with Fact_invoice
Fact_invoice['invoicedate']=pd.to_datetime(Fact_invoice['invoicedate']).dt.tz_localize(None)
Fact_invoice.head()

,invoiceid,customerid,invoicedate,billingaddress,billingcity,billingstate,billingcountry,billingpostalcode,total,last_update
0,1,2,2018-01-01,Theodor-Heuss-Straße 34,Stuttgart,NaN,Germany,70174,1.98,2023-01-04 12:05:40.330691
1,2,4,2018-01-02,Ullevålsveien 14,Oslo,NaN,Norway,0171,3.96,2023-01-04 12:05:40.332702
2,3,8,2018-01-03,Grétrystraat 63,Brussels,NaN,Belgium,1000,5.94,2023-01-04 12:05:40.334024
3,4,14,2018-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,2023-01-04 12:05:40.335723
4,5,23,2018-01-11,69 Salem Street,Boston,MA,USA,2113,13.86,2023-01-04 12:05:40.336992


In [ ]:
Invoice_change=pd.merge(Fact_invoice, Dim_currency, left_on='invoicedate',right_on='invoicedate', how='left')
Invoice_change.head()

,invoiceid,customerid,invoicedate,billingaddress,billingcity,billingstate,billingcountry,billingpostalcode,total,last_update,USD_to_NIS
0,1,2,2018-01-01,Theodor-Heuss-Straße 34,Stuttgart,NaN,Germany,70174,1.98,2023-01-04 12:05:40.330691,3.475750
1,2,4,2018-01-02,Ullevålsveien 14,Oslo,NaN,Norway,0171,3.96,2023-01-04 12:05:40.332702,3.450670
2,3,8,2018-01-03,Grétrystraat 63,Brussels,NaN,Belgium,1000,5.94,2023-01-04 12:05:40.334024,3.449760
3,4,14,2018-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,2023-01-04 12:05:40.335723,3.433863
4,5,23,2018-01-11,69 Salem Street,Boston,MA,USA,2113,13.86,2023-01-04 12:05:40.336992,3.423510


In [ ]:
Invoice_change["Total_nis"]=Invoice_change["total"]*Invoice_change["USD_to_NIS"] #Get the total in NIS
Invoice_change.head()

,invoiceid,customerid,invoicedate,billingaddress,billingcity,billingstate,billingcountry,billingpostalcode,total,last_update,USD_to_NIS,Total_nis
0,1,2,2018-01-01,Theodor-Heuss-Straße 34,Stuttgart,NaN,Germany,70174,1.98,2023-01-04 12:05:40.330691,3.475750,6.881985
1,2,4,2018-01-02,Ullevålsveien 14,Oslo,NaN,Norway,0171,3.96,2023-01-04 12:05:40.332702,3.450670,13.664653
2,3,8,2018-01-03,Grétrystraat 63,Brussels,NaN,Belgium,1000,5.94,2023-01-04 12:05:40.334024,3.449760,20.491574
3,4,14,2018-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,2023-01-04 12:05:40.335723,3.433863,30.595719
4,5,23,2018-01-11,69 Salem Street,Boston,MA,USA,2113,13.86,2023-01-04 12:05:40.336992,3.423510,47.449849


In [ ]:
Total_sum = Invoice_change.groupby('customerid').agg(total_usd_sum=('total', 'sum'),total_nis_sum=('Total_nis', 'sum')).reset_index().sort_values(by='total_usd_sum', ascending=False).head(5)
#Get the top 5 customers by total sum
Total_sum

,customerid,total_usd_sum,total_nis_sum
58,59,71.31,245.839280
131,132,65.36,226.709967
178,179,55.44,186.038187
5,6,49.62,171.941778
25,26,47.62,157.982582


In [ ]:
Top_5_customer=pd.merge(Total_sum,Dim_customer,how="left",on="customerid")
Top_5_customer=Top_5_customer[["customerid","firstname","lastname","total_usd_sum","total_nis_sum"]] #Get the names of the best customers and only the columns that we need
Top_5_customer


,customerid,firstname,lastname,total_usd_sum,total_nis_sum
0,59,Puja,Srivastava,71.31,245.839280
1,132,Denise,Kelly,65.36,226.709967
2,179,Josephine,Gomez,55.44,186.038187
3,6,Helena,Holý,49.62,171.941778
4,26,Richard,Cunningham,47.62,157.982582
